In [ ]:
import numpy as np
import random
from sklearn.datasets import load_iris
from collections import Counter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_iris,load_wine,load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier

In [ ]:
class Node:

    def __init__(self,id=0):
        self.id = id
        self.connections = []
        self.value = random.uniform(0,1) < 0.5
        self.probabilities = []
        

    def add_connection(self,node):
        rnum = random.uniform(0,1)
        if rnum < 0.25:
            if node.id not in self.connections:
                self.connections.append(node)
                if len(self.probabilities) > 0:
                    ratio = sum(self.probabilities) / len(self.probabilities)
                    if ratio == 0:
                        ratio = 0.1
                    self.probabilities.append(ratio)
                    self.arrange_probabilities()
                else:
                    self.probabilities.append(0.1)
            else:
                print("The node is still in connections")
        else:
            pass

    def arrange_probabilities(self):
        self.probabilities = self.probabilities[0:len(self.connections)]
        s_arr = sum(self.probabilities)
        if s_arr < 1:
            self.probabilities.append(1-s_arr)
        else:
            self.probabilities = [i/s_arr for i in self.probabilities]


    def calculate_value(self):  
        if len(self.connections) >3:
            node = np.random.choice(self.connections, p=self.probabilities)
            self.value = node.value
        else:
            pass

    def print_values(self):
        print(f"the node has {len(self.connections)} number of connections")
        for node in self.connections:
            print(f"the connection is {node.id}")
            print(f"the connection's value is {node.value}")
        
        print(f"MAIN value of the node {self.id} is {self.value}")

        print(f"probability values are {self.probabilities}")

        

In [3]:
nodes = [Node(i) for i in range(100) ]

In [4]:
for _ in range(2000):
    node_main = random.choice(nodes)
    node_other = random.choice(nodes)
    if node_main.id == node_other.id:
        continue
    node_main.add_connection(node_other)
    
    node_main.calculate_value()

    
    

In [5]:
node_main.print_values()


the node has 6 number of connections
the connection is 40
the connection's value is True
the connection is 25
the connection's value is True
the connection is 5
the connection's value is False
the connection is 39
the connection's value is True
the connection is 69
the connection's value is False
the connection is 39
the connection's value is True
MAIN value of the node 76 is False
probability values are [0.05, 0.05, 0.4, 0.16666666666666666, 0.16666666666666669, 0.16666666666666669]


In [6]:
class LearnerNode:

    def __init__(self, id=0,row=[],target=""):
        self.id = id
        self.nodes = []
        self.distance_val = 1
        self.value = target
        self.row  = row

    
    def select_related_nodes(self,node):
        vote = 0
        for row in self.nodes:
            if self.calculate_distance_row(row,node.row) < self.distance_val:
                vote += 1

        if vote > len(self.nodes) / 2:
            self.nodes.append(node)
        
    def calculate_distance_row(self,a,b):
        abs_dist = [abs(i-j) for i,j in zip(a,b)]  
        return sum(abs_dist)

    def assing_distance(self):
        distance_values = []
        if len(self.nodes) > 0:
            for node in self.nodes:
                distance_values.append(self.calculate_distance_row(self.row,node.row))
            
            self.distance_val = min(distance_values)


    



    

        

    

In [7]:
data = load_iris()

In [8]:
X = data.data
y = data.target

In [9]:
class LearningUnits:

    def __init__(self,data,target):
        self.data = data
        self.target = target
        self.pool_of_learners = []
        


    def collect_learners(self):

        for i,row in enumerate(self.data):
            learner = LearnerNode(id=i,row=row,target=self.target[i])
            self.pool_of_learners.append(learner)
            for j in range(i):
                learner.select_related_nodes(self.pool_of_learners[j])

            learner.assing_distance()

    def selected_dist_group(self,row):
        selected_dist_group = []
        for learner in self.pool_of_learners:
            dist_temp = learner.calculate_distance_row(row,learner.row)
            if dist_temp <= learner.distance_val:
                selected_dist_group.append((dist_temp,learner))
                
        if len(selected_dist_group) > 0:
            return selected_dist_group 
        else:
            dist_per = np.inf
            for learner in self.pool_of_learners:
                dist_temp = learner.calculate_distance_row(row,learner.row)
                if dist_temp < dist_per:
                    dist_per = dist_temp
                    selected_dist_group.append((dist_temp,learner))

            return selected_dist_group[-3::]
        

    def predict(self,row):
        selected_rows = self.selected_dist_group(row)
        #print(selected_rows)
        selected_row = np.inf
        selected_value = None
        for row in selected_rows:
            if row[0] < selected_row:
                selected_row = row[0]
                selected_value = row[1].value
        return selected_value




            






In [10]:
agent_learning = LearningUnits(data=X,target=y)
agent_learning.collect_learners()
test = X[0]
test_result = y[0]
test_val = agent_learning.predict(test)
print(test_result,test_val)

0 0


In [11]:
def calculate_learner_accuracy(agent_learning,X,y):
    error = 0
    correct = 0
    print(X,y)
    for x_row,y_target in zip(X,y):
        prediction = agent_learning.predict(x_row)
        if prediction[0][0] == y_target:
            correct += 1
            error += 1


    print(f"correct num: {correct}, error num: {error} ratio: {correct/(error + correct)}")


In [12]:

iris = load_iris()

X = iris.data
y = iris.target

feature_names = iris.feature_names
target_names = iris.target_names

df = pd.DataFrame(X, columns=feature_names)
df["species"] = y


model = LogisticRegression(max_iter=200)
model.fit(X, y)

y_pred = model.predict(X)

accuracy = accuracy_score(y, y_pred)
print("Accuracy:", accuracy)

print("\nConfusion Matrix:")
print(confusion_matrix(y, y_pred))

print("\nClassification Report:")
print(classification_report(y, y_pred, target_names=target_names))



Accuracy: 0.9733333333333334

Confusion Matrix:
[[50  0  0]
 [ 0 47  3]
 [ 0  1 49]]

Classification Report:
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        50
  versicolor       0.98      0.94      0.96        50
   virginica       0.94      0.98      0.96        50

    accuracy                           0.97       150
   macro avg       0.97      0.97      0.97       150
weighted avg       0.97      0.97      0.97       150



## Test with Another Data

In [13]:
data = pd.read_csv("Heart_Disease_Prediction.csv")

data.columns

Index(['Age', 'Sex', 'Chest pain type', 'BP', 'Cholesterol', 'FBS over 120',
       'EKG results', 'Max HR', 'Exercise angina', 'ST depression',
       'Slope of ST', 'Number of vessels fluro', 'Thallium', 'Heart Disease'],
      dtype='str')

In [14]:
data.head()

,Age,Sex,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,Max HR,Exercise angina,ST depression,Slope of ST,Number of vessels fluro,Thallium,Heart Disease
0,70,1,4,130,322,0,2,109,0,2.4,2,3,3,Presence
1,67,0,3,115,564,0,2,160,0,1.6,2,0,7,Absence
2,57,1,2,124,261,0,0,141,0,0.3,1,0,7,Presence
3,64,1,4,128,263,0,0,105,1,0.2,2,1,7,Absence
4,74,0,2,120,269,0,2,121,1,0.2,1,1,3,Absence


In [15]:
def calculate_learner_accuracy(agent_learning,X,y):
    error = 0
    correct = 0
    for x_row,y_target in zip(X,y):
        prediction = agent_learning.predict(x_row)
        #print(prediction,y_target)
        if prediction == y_target:
            correct += 1
        else:
            error += 1


    print(f"correct num: {correct}, error num: {error} ratio: {correct/(error + correct)}")


In [16]:
X = data.drop('Heart Disease',axis=1).values
y = data['Heart Disease'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
agent_learning = LearningUnits(data=X_train,target=y_train)
agent_learning.collect_learners()
calculate_learner_accuracy(agent_learning,X=X_test,y=y_test)

correct num: 35, error num: 19 ratio: 0.6481481481481481


In [18]:
kmeans = KMeans(
    n_clusters=3,
    random_state=42,
    n_init=10
)

kmeans.fit(X_train,y_train)
y_pred = kmeans.predict(X_test)
y = pd.Categorical(y)


In [19]:
def match_predict(predict,real):
    correct = 0
    error = 0
    for p,r in zip(predict,real):
        if p == r:
            correct += 1
        else:
            error+=1
    
    print(f"correct num: {correct}, error num: {error} ratio: {correct/(error + correct)}")


In [20]:
match_predict(y_pred,y.codes)

correct num: 22, error num: 32 ratio: 0.4074074074074074


In [21]:
rf_classifier = RandomForestClassifier()

rf_classifier.fit(X_train,y_train)
y_pred = rf_classifier.predict(X_test)
y = pd.Categorical(y)


match_predict(y_pred,y)


correct num: 28, error num: 26 ratio: 0.5185185185185185


# Wine DataSet Test

In [22]:
data = load_wine()

In [23]:


X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
agent_learning = LearningUnits(data=X_train,target=y_train)
agent_learning.collect_learners()
calculate_learner_accuracy(agent_learning,X=X_test,y=y_test)

correct num: 30, error num: 6 ratio: 0.8333333333333334


In [25]:
rf_classifier = RandomForestClassifier()

rf_classifier.fit(X_train,y_train)
y_pred = rf_classifier.predict(X_test)
y = pd.Categorical(y)


match_predict(y_pred,y)


correct num: 14, error num: 22 ratio: 0.3888888888888889


# Kaggle Credit Score Dataset

In [26]:
data_train = pd.read_csv('train.csv')

C:\Users\brs_c\AppData\Local\Temp\ipykernel_13096\238125259.py:1: DtypeWarning: Columns (0: Monthly_Balance) have mixed types. Specify dtype option on import or set low_memory=False.
  data_train = pd.read_csv('train.csv')


In [27]:
data_train.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good


In [28]:
data_train.info()

<class 'pandas.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  str    
 1   Customer_ID               100000 non-null  str    
 2   Month                     100000 non-null  str    
 3   Name                      90015 non-null   str    
 4   Age                       100000 non-null  str    
 5   SSN                       100000 non-null  str    
 6   Occupation                100000 non-null  str    
 7   Annual_Income             100000 non-null  str    
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  str    
 13  Type_of_Loan              88592 non-null   str    
 14  

In [29]:
def turn_data_into_codes(data):
    data = data.copy()
    for col in data.columns:
        if data[col].dtype == "str" or data[col].dtype ==  "object":
            print(col)
            data[col] = data[col].astype("category").cat.codes
    return data


def drop_non_float_rows(df):
    mask = df.apply(
        lambda row: pd.to_numeric(row, errors="coerce").notna().all(),
        axis=1
    )
    return df[mask]



In [30]:
data_train = turn_data_into_codes(data_train)
data_train

data_train = drop_non_float_rows(data_train)

ID
Customer_ID
Month
Name
Age
SSN
Occupation
Annual_Income
Num_of_Loan
Type_of_Loan
Num_of_Delayed_Payment
Changed_Credit_Limit
Credit_Mix
Outstanding_Debt
Credit_History_Age
Payment_of_Min_Amount
Amount_invested_monthly
Payment_Behaviour
Monthly_Balance
Credit_Score


In [31]:
data_train.dtypes

ID                            int32
Customer_ID                   int16
Month                          int8
Name                          int16
Age                           int16
SSN                           int16
Occupation                     int8
Annual_Income                 int16
Monthly_Inhand_Salary       float64
Num_Bank_Accounts             int64
Num_Credit_Card               int64
Interest_Rate                 int64
Num_of_Loan                   int16
Type_of_Loan                  int16
Delay_from_due_date           int64
Num_of_Delayed_Payment        int16
Changed_Credit_Limit          int16
Num_Credit_Inquiries        float64
Credit_Mix                     int8
Outstanding_Debt              int16
Credit_Utilization_Ratio    float64
Credit_History_Age            int16
Payment_of_Min_Amount          int8
Total_EMI_per_month         float64
Amount_invested_monthly       int32
Payment_Behaviour              int8
Monthly_Balance               int32
Credit_Score                

In [32]:
data_train_sample = data_train.sample(300)

X = data_train_sample.drop('Credit_Score',axis=1).values
y = data_train_sample['Credit_Score'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:
rf_classifier = RandomForestClassifier()

rf_classifier.fit(X_train,y_train)
y_pred = rf_classifier.predict(X_test)

match_predict(y_pred,y_test)


correct num: 35, error num: 25 ratio: 0.5833333333333334


In [34]:
agent_learning = LearningUnits(data=X_train,target=y_train)
agent_learning.collect_learners()
calculate_learner_accuracy(agent_learning,X=X_test,y=y_test)

correct num: 34, error num: 26 ratio: 0.5666666666666667


# Breast Cancer DataSet

In [35]:
data = load_breast_cancer()

X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [36]:
agent_learning = LearningUnits(data=X_train,target=y_train)
agent_learning.collect_learners()
calculate_learner_accuracy(agent_learning,X=X_test,y=y_test)

correct num: 106, error num: 8 ratio: 0.9298245614035088


In [37]:
rf_classifier = RandomForestClassifier()

rf_classifier.fit(X_train,y_train)
y_pred = rf_classifier.predict(X_test)

match_predict(y_pred,y_test)

correct num: 110, error num: 4 ratio: 0.9649122807017544


In [38]:
rf_classifier = KMeans(n_clusters=len(np.unique(y)), random_state=0, n_init="auto")
rf_classifier.fit(X_train,y_train)
y_pred = rf_classifier.predict(X_test)

match_predict(y_pred,y_test)

correct num: 100, error num: 14 ratio: 0.8771929824561403
